In [1]:
from __future__ import division

from staintools import ReinhardNormalizer
from staintools import MacenkoNormalizer
from staintools import VahadaneNormalizer

from staintools import standardize_brightness
from staintools.utils.visual import read_image, show, show_colors, build_stack, patch_grid

import os
import cv2
from multiprocessing import cpu_count
from concurrent.futures import ProcessPoolExecutor, as_completed

In [2]:
def scan_files(directory, prefix=None, postfix=None):
    files_list = []
    for root, sub_dirs, files in os.walk(directory):
        for special_file in files:
            if postfix:
                if special_file.endswith(postfix):
                    files_list.append(os.path.join(root, special_file))
            elif prefix:
                if special_file.startswith(prefix):
                    files_list.append(os.path.join(root, special_file))
            else:
                files_list.append(os.path.join(root, special_file))
    return files_list

In [3]:
def process_image(normalizer, image_in, save_dir, depth):
    tokens = image_in.rsplit(os.sep, depth+1)
    image_out = os.path.join(save_dir, *tokens[1:])
    parent_dir = os.path.dirname(image_out)
    os.makedirs(parent_dir, exist_ok=True)
    r_img = read_image(image_in)
    target = normalizer.transform(r_img)
    target = cv2.cvtColor(target, cv2.COLOR_RGB2BGR)
    cv2.imwrite(image_out, target)
    
def batch_process_image(images_in, save_dir, depth):
    normalizer = VahadaneNormalizer()
    i1 = read_image("/home/sakulaki/yolo-yuli/tct/train_c1/StainTools/lbp_pic/reference_pic/zs_abnormal.png")
    normalizer.fit(i1)
    
    for image_in in images_in:
        process_image(normalizer, image_in, save_dir, depth)

In [4]:
def main(input_dir, output_dir, depth):
    images_in = scan_files(input_dir, postfix=".jpg")
    print("total of {} images to process".format(len(images_in)))
    
    executor = ProcessPoolExecutor(max_workers=cpu_count()-4)
    tasks = []
    
    interval = 100
    for i in range(0, len(images_in), interval):
        batch = images_in[i : i+interval]
#         batch_process_image(images_in=batch, save_dir=output_dir, depth=depth)
        tasks.append(executor.submit(batch_process_image, batch, output_dir, depth))
        print("added {} - {} to process pool".format(i, i+interval))
    
    job_count = len(tasks)
    for future in as_completed(tasks):
        job_count -= 1
        print("processed {} images, remaining job count: {}".format(interval, job_count))

In [5]:
input_dir = "/home/sakulaki/yolo-yuli/tct/train_c1/algorithm/arrange_patches_12_Glandular"
output_dir = "/home/sakulaki/yolo-yuli/tct/train_c1/algorithm/arrange_patches_12_Glandular_new"
depth = 2  # preserve directory tree from input_dir to output_dir, up to depth

In [6]:
main(input_dir, output_dir, depth)

total of 31645 images to process
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
added 0 - 100 to process pool
added 100 - 200 to process pool
added 200 - 300 to process pool
added 300 - 400 to process pool
added 400 - 500 to process pool
added 500 - 600 to process pool
added 600 - 700 to process pool
added 700 - 800 to process pool
added 800 - 900 to process pool
added 900 - 1000 to process pool
added 1000 - 1100 to process pool
added 1100 - 1200 to process pool
added 1200 - 1300 to process pool
added 1300 - 1400 to process pool
added 1400 - 1500 to process pool
added 1500 - 1600 to process pool
added 1600 - 1700 to process pool
added 1700 - 1800 to process pool
added 1800 - 1900 to process pool
added 1900 -

Using brightness standardization
processed 100 images, remaining job count: 316
Using brightness standardization
processed 100 images, remaining job count: 315
Using brightness standardization
processed 100 images, remaining job count: 314
Using brightness standardization
processed 100 images, remaining job count: 313
Using brightness standardization
processed 100 images, remaining job count: 312
Using brightness standardization
processed 100 images, remaining job count: 311
Using brightness standardization
processed 100 images, remaining job count: 310
Using brightness standardization
processed 100 images, remaining job count: 309
Using brightness standardization
Using brightness standardization
processed 100 images, remaining job count: 308
processed 100 images, remaining job count: 307
Using brightness standardization
processed 100 images, remaining job count: 306
Using brightness standardization
processed 100 images, remaining job count: 305
Using brightness standardization
process

processed 100 images, remaining job count: 214
Using brightness standardization
processed 100 images, remaining job count: 213
Using brightness standardization
processed 100 images, remaining job count: 212
Using brightness standardization
processed 100 images, remaining job count: 211
Using brightness standardization
processed 100 images, remaining job count: 210
Using brightness standardization
processed 100 images, remaining job count: 209
Using brightness standardization
processed 100 images, remaining job count: 208
Using brightness standardization
processed 100 images, remaining job count: 207
Using brightness standardization
processed 100 images, remaining job count: 206
Using brightness standardization
processed 100 images, remaining job count: 205
Using brightness standardization
processed 100 images, remaining job count: 204
Using brightness standardization
processed 100 images, remaining job count: 203
Using brightness standardization
processed 100 images, remaining job coun

Using brightness standardization
processed 100 images, remaining job count: 111
Using brightness standardization
processed 100 images, remaining job count: 110
Using brightness standardization
processed 100 images, remaining job count: 109
Using brightness standardization
processed 100 images, remaining job count: 108
Using brightness standardization
processed 100 images, remaining job count: 107
Using brightness standardization
processed 100 images, remaining job count: 106
Using brightness standardization
processed 100 images, remaining job count: 105
Using brightness standardization
processed 100 images, remaining job count: 104
Using brightness standardization
processed 100 images, remaining job count: 103
Using brightness standardization
processed 100 images, remaining job count: 102
Using brightness standardization
processed 100 images, remaining job count: 101
Using brightness standardization
processed 100 images, remaining job count: 100
Using brightness standardization
process

processed 100 images, remaining job count: 6
processed 100 images, remaining job count: 5
processed 100 images, remaining job count: 4
processed 100 images, remaining job count: 3
processed 100 images, remaining job count: 2
processed 100 images, remaining job count: 1
processed 100 images, remaining job count: 0
